In [143]:
#PART 5
#ORGANIZING CODE ACCORDING TO THE GIVEN UML

import math
from abc import ABC, abstractmethod

class MinHeap:
    def __init__(self, data):
        self.items = data
        self.length = len(data)
        self.build_heap()
        self.map = {}
        for i in range(self.length):
            self.map[self.items[i].value] = i

    def find_left_index(self,index):
        return 2 * (index + 1) - 1

    def find_right_index(self,index):
        return 2 * (index + 1)

    def find_parent_index(self,index):
        return (index + 1) // 2 - 1  

    def sink_down(self, index):
        smallest_known_index = index
        left_index = self.find_left_index(index)
        right_index = self.find_right_index(index)
        if left_index < self.length and self.items[left_index].key < self.items[smallest_known_index].key:
            smallest_known_index = left_index

        if right_index < self.length and self.items[right_index].key < self.items[smallest_known_index].key:
            smallest_known_index = right_index

        if smallest_known_index != index:
            self.items[index], self.items[smallest_known_index] = self.items[smallest_known_index], self.items[index]
            
            self.map[self.items[index].value] = index
            self.map[self.items[smallest_known_index].value] = smallest_known_index

            self.sink_down(smallest_known_index)

    def build_heap(self):
        for i in range(self.length // 2 - 1, -1, -1):
            self.sink_down(i) 

    def insert(self, node):
        self.items.append(node)
        self.map[node.value] = self.length
        self.length += 1
        self.swim_up(self.length - 1)

    def swim_up(self, index):
        while index > 0 and self.items[index].key < self.items[self.find_parent_index(index)].key:
            parent_index = self.find_parent_index(index)
            self.items[index], self.items[parent_index] = self.items[parent_index], self.items[index]
            #update map
            self.map[self.items[index].value] = index
            self.map[self.items[parent_index].value] = parent_index
            index = parent_index

    def extract_min(self):
        if self.length == 0:
            return None
        self.items[0], self.items[self.length - 1] = self.items[self.length - 1], self.items[0]
        self.map[self.items[self.length - 1].value] = self.length - 1
        self.map[self.items[0].value] = 0

        min_node = self.items.pop()
        self.length -= 1
        self.map.pop(min_node.value, None)
        if self.length > 0:
            self.sink_down(0)
        return min_node

    def decrease_key(self, value, new_key):
        if value in self.map and new_key < self.items[self.map[value]].key:
            index = self.map[value]
            self.items[index].key = new_key
            self.swim_up(index)

    def is_empty(self):
        return self.length == 0
    
    def find_item(self, k):

        for i in self.items:
            if i.key == k:
                return True
        
        return False

    def __str__(self):
        height = math.ceil(math.log(self.length + 1, 2))
        whitespace = 2 ** height + height
        s = ""
        for i in range(height):
            for j in range(2 ** i - 1, min(2 ** (i + 1) - 1, self.length)):
                s += " " * whitespace
                s += str(self.items[j]) + " "
            s += "\n"
            whitespace = whitespace // 2
        return s

class Item:
    def __init__(self, value, key):
        self.key = key
        self.value = value
    
    def __str__(self):
        return "(" + str(self.key) + "," + str(self.value) + ")"


class Graph(ABC):
    def __init__(self, nodes):
        self.graph = [[] for _ in range(nodes)]

    def add_node(self, node):
        if node >= len(self.graph):
            self.graph.append([])

    def get_adj_nodes(self, node):
        return self.graph[node]

    @abstractmethod
    def w(self, node):
        pass

    def get_number_of_nodes(self):
        return len(self.graph)

    def get_nodes(self):
        return range(len(self.graph))

class WeightedGraph(Graph):
    def __init__(self, nodes):
        super().__init__(nodes)
        self.weights = {}

    def add_edge(self, node1, node2, weight):
        if node2 not in self.graph[node1]:
            self.graph[node1].append(node2)
        self.weights[(node1, node2)] = weight

    def w(self, node1, node2):
        return self.weights.get((node1, node2), float('inf'))

    def are_connected(self, node1, node2):
        return node2 in self.get_adj_nodes(node1)


class HeuristicGraph(WeightedGraph):
    def __init__(self, nodes, target_node, x_coor, y_coor):
        super().__init__(nodes)
        self.heuristic = {}
        
        for node in range(self.get_number_of_nodes()):
            x1 = x_coor[node]
            y1 = y_coor[node]
            x2 = x_coor[target_node]
            y2 = y_coor[target_node]
            dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            self.heuristic[node] = dist

    def get_heuristic(self, node=None):
        if node is not None:
            return self.heuristic.get(node)
        return self.heuristic


class SPAgorithm(ABC):
    @abstractmethod
    def calc_sp(self, graph: Graph, source: int, dest: int) -> float:
        pass

# Implementation of Dijkstra's algorithm as SPAgorithm
class Dijkstra(SPAgorithm):
    def __init__(self, graph):
        self.graph = graph
        self.node_count = graph.get_number_of_nodes()
        self.distances = {node: float('inf') for node in range(self.node_count)}
        self.paths = {node: [] for node in range(self.node_count)}
    
    def find_shortest_paths(self, source):
        # Initialize distances and paths for the source
        self.distances[source] = 0
        self.paths[source] = [source]
        
        # Initialize MinHeap
        initialize_heap = [Item(node, self.distances[node]) for node in range(self.node_count)]
        self.min_heap = MinHeap(initialize_heap)
        
        # Process the nodes
        while not self.min_heap.is_empty():
            current_node = self.min_heap.extract_min()
            node_index = current_node.key
            
            for neighbor in self.graph.get_neighbors(node_index):
                weight = self.graph.get_weights(node_index, neighbor)
                new_distance = self.distances[node_index] + weight
                
                if new_distance < self.distances[neighbor]:
                    self.distances[neighbor] = new_distance
                    self.paths[neighbor] = self.paths[node_index] + [neighbor]
                    self.min_heap.decrease_key(neighbor, new_distance)
        
        return self.distances, self.paths

# Implementation of Bellman-Ford algorithm as SPAgorithm
class BellmanFord(SPAgorithm):
    def __init__(self,g,src):
        self.g = g
        self.src = src
        self.dist = {}
        self.path = {}
        
    def startvals(self):
        for i in range(g.get_number_of_nodes()):
            self.dist[i] = float("inf")
            self.path[i] = [self.src]
            
        self.dist[self.src] = 0
        self.path[self.src] = [self.src]
    
    def code(self):
        
        for _ in range(len(self.g.get_nodes()) - 1):
            difference = False
            for node in self.g.get_nodes():
                for n in self.g.get_neighbors(node):
                    weight = self.g.get_weights(node,n)
                    if weight != None:
                        if self.dist[node] + weight < self.dist[n]:
                            self.dist[n] = self.dist[node] + weight
                            self.path[n] = self.path[node] + [n]
                            difference = True
            if not difference:
                return self.dist,self.path
            
    def get_distance(self):
        return self.dist
    
    def get_path(self):
        return self.path

class A_Star(SPAgorithm):

    def __init__(self, G: WeightedGraph, source, destination, h): 

        self.G = G
        self.source = source
        self.destination = destination
        self.h = h

        self.g = {}
        self.f = {}

        self.pred = {}
        self.path = []

    def make_path(self):

        pred = self.pred
        dest = self.destination
        s = self.source


        path = []

        curr = dest

        while pred[curr] != None:

            path = [curr] + path

            curr = pred[curr]

        path = [s] + path

        #print(path)

        return path

    def shortest_path(self):

        graph = self.G.graph

        #initialize open list and put start/source node in it and its initial f of 0
        open = MinHeap([])
        open.insert(Item(0,self.source))

        #initialize closed list
        closed = []

        #initialize predessor dict
        self.pred = {self.source: None}

        #initialize all node g and f values to be infinity

        #g
        for node in range(len(graph)):
            self.g[node] = float('inf')

        self.g[self.source] = 0

        #f
        for node in range(len(graph)):
            self.f[node] = float('inf')


        self.f[self.source] = self.h[self.source]

        #search for shortest path
        while open.length > 0:

           # print("open=",open.map)
            curr_f = open.extract_min() #(f, node) 
            curr = curr_f.value
           # print("curr=",curr_f)

            if curr == self.destination:
                
                self.path = self.make_path()
                return (self.pred, self.path) 

            #iterate through the current nodes neighbors
            for node in graph[curr]:
                #print("neighbor=",node)
                if node == self.destination:
                    self.pred[node] = curr

                    self.path = self.make_path()
                    return (self.pred, self.path) 


                new_g = self.g[curr] + self.G.get_weights(curr,node)

                if new_g < self.g[node]:

                    self.pred[node] = curr

                    self.g[node] = new_g
                    self.f[node] = new_g + self.h[node]


                    if open.find_item(node) and node not in closed:
                        open.decrease_key(self.f[node],node)
                    elif not open.find_item(node) and node not in closed:
                        open.insert(Item(self.f[node],node))

                        #print("open=",open.map)


            #add curr to closed
            closed.append(curr)
    
    
        self.path = self.make_path()
        return (self.pred, self.path) 
    
    def get_g(self):
        return self.g
    
    def get_f(self):
        return self.f
    
    def get_predecessors(self):
        return self.pred
    
    def get_shortest_path(self):
        return self.path

class ShortPathFinder:
    def __init__(self):
        self.graph = None
        self.algorithm = None

    def calc_short_path(self, source: int, dest: int) -> float:
        if not self.graph or not self.algorithm:
            raise ValueError("Set graph and algorithm.")
        return self.algorithm.calc_sp(self.graph, source, dest)

    def set_graph(self, graph: Graph):
        self.graph = graph

    def set_algorithm(self, algorithm: SPAgorithm):
        self.algorithm = algorithm